In [1]:
from datetime import datetime, timedelta
from io import BytesIO, TextIOWrapper
import tarfile
from urllib.request import urlopen

import pandas as pd
import pytz

We'll define a function `get_updates` that takes a `datetime` argument and returns a pandas DataFrame containing all reported vehicle updates for that day (assuming they're available). You can also visit http://mbta-history.apptic.xyz directly and download the data for a particular day.

In [2]:
def get_updates(dt):
    # This reads the gzipped tarfile into memory, extracts the file,
    # and reads it directly into a dataframe.
    
    # Pandas can also read directly from URLs and file paths
    csv_filename = yesterday.strftime("%Y-%m-%d.csv")
    urlpath = dt.strftime("http://mbta-history.apptic.xyz/%m/" + csv_filename + ".tgz")
    filedata = BytesIO(urlopen(urlpath).read())
    tar = tarfile.open(mode="r:gz", fileobj=filedata)
    return pd.read_csv(tar.extractfile(csv_filename))

And let's retrieve the updates for yesterday:

In [3]:
now_utc = pytz.utc.localize(datetime.utcnow())
yesterday = now_utc.astimezone(pytz.timezone("US/Eastern")) - timedelta(days=1)

df = get_updates(yesterday)
df.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,status,timestamp,lat,lon
0,35241148,2017-09-13,339,5,y1855,STOPPED_AT,2017-09-13 07:29:44,42.292591,-71.062347
1,35241149,2017-09-13,393,26,y1915,STOPPED_AT,2017-09-13 07:29:59,42.320618,-71.081642
2,35241148,2017-09-13,325,10,y1855,STOPPED_AT,2017-09-13 07:30:44,42.298000,-71.060829
3,35241149,2017-09-13,21151,30,y1915,INCOMING_AT,2017-09-13 07:30:59,42.324902,-71.083038
4,35241149,2017-09-13,396,29,y1915,STOPPED_AT,2017-09-13 07:31:13,42.326279,-71.083290


Note that these are trips *originating* yesterday. They can actually arrive at stops on a different calendar day, if they run past midnight. But their scheduled stop times are anchored to the "trip_start" date, as we'll see shortly.

Now let's massage the data a bit.

CSV doesn't have any special handling for dates, so we need to convert the timestamps to datetimes that can be used in comparisons.

The recorded timestamps are actually UTC. UTC doesn't observe DST, so a UTC timestamp unambiguously identifies a moment in time. Initially, though, we're just going to use "naive" datetimes. We'll convert them to US/Eastern later.

In [4]:
df.timestamp = pd.to_datetime(df.timestamp)

We'll take the max timestamp for each trip stop. We could filter for rows where `status == "STOPPED_AT"`, but there is not a record for every status change for every stop for every trip. (There is not even a record for every stop for every trip!) So we just have to make do with the last timestamp.

In [5]:
df["max_timestamp"] = df.groupby(["trip_id", "stop_sequence"]).timestamp.transform("max")
df.head(5)

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,status,timestamp,lat,lon,max_timestamp
0,35241148,2017-09-13,339,5,y1855,STOPPED_AT,2017-09-13 07:29:44,42.292591,-71.062347,2017-09-13 07:29:44
1,35241149,2017-09-13,393,26,y1915,STOPPED_AT,2017-09-13 07:29:59,42.320618,-71.081642,2017-09-13 07:29:59
2,35241148,2017-09-13,325,10,y1855,STOPPED_AT,2017-09-13 07:30:44,42.298000,-71.060829,2017-09-13 07:30:44
3,35241149,2017-09-13,21151,30,y1915,INCOMING_AT,2017-09-13 07:30:59,42.324902,-71.083038,2017-09-13 07:30:59
4,35241149,2017-09-13,396,29,y1915,STOPPED_AT,2017-09-13 07:31:13,42.326279,-71.083290,2017-09-13 07:31:13


Now, some minor cleanup. Remove duplicate entries and get rid of some columns we won't be using. Convert the timestamps to Eastern time.

(Note that this would not be wise if we were working with data from multiple days, since `trip_id` is unique on a given day, not across days.)

In [7]:
stops = df.drop_duplicates(subset=["trip_id", "stop_sequence"])
del stops["timestamp"]
del stops["status"]
stops.rename(columns={"max_timestamp": "timestamp"}, inplace=True)
stops["timestamp"] = stops.timestamp.dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
stops.head()

/Users/brian/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/Users/brian/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp
0,35241148,2017-09-13,339,5,y1855,42.292591,-71.062347,2017-09-13 03:29:44-04:00
1,35241149,2017-09-13,393,26,y1915,42.320618,-71.081642,2017-09-13 03:29:59-04:00
2,35241148,2017-09-13,325,10,y1855,42.298000,-71.060829,2017-09-13 03:30:44-04:00
3,35241149,2017-09-13,21151,30,y1915,42.324902,-71.083038,2017-09-13 03:30:59-04:00
4,35241149,2017-09-13,396,29,y1915,42.326279,-71.083290,2017-09-13 03:31:13-04:00


Remove the commuter rail and ferries:

In [8]:
# remove commuter rail and ferries
stops = stops[~(stops.trip_id.str.contains("CR") | stops.trip_id.str.startswith("Boat"))]
stops.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp
0,35241148,2017-09-13,339,5,y1855,42.292591,-71.062347,2017-09-13 03:29:44-04:00
1,35241149,2017-09-13,393,26,y1915,42.320618,-71.081642,2017-09-13 03:29:59-04:00
2,35241148,2017-09-13,325,10,y1855,42.298000,-71.060829,2017-09-13 03:30:44-04:00
3,35241149,2017-09-13,21151,30,y1915,42.324902,-71.083038,2017-09-13 03:30:59-04:00
4,35241149,2017-09-13,396,29,y1915,42.326279,-71.083290,2017-09-13 03:31:13-04:00


We're more interested in **routes** than in **trips**. The terminology is explained thoroughly in the [GTFS Reference](https://developers.google.com/transit/gtfs/reference/). Briefly, a trip is a collection of stops and (relative) times associated with a schedule and a route. The schedule determines the days when the trip runs. So, the Red line train to Ashmont leaving Alewife on non-holiday weekdays at 6:00 AM is a trip.

In order to connect our trip_ids to route_ids, we have to join our vehicle update data to the static data provided by the MBTA. You can download the full manifest [here](http://www.mbta.com/uploadedfiles/MBTA_GTFS.zip).

We're just going to load the zip in memory and extract what we need.

To that end, some functions:

In [10]:
from zipfile import ZipFile

def get_manifest(url="http://www.mbta.com/uploadedfiles/MBTA_GTFS.zip"):
    with urlopen(url) as u:
        return ZipFile(BytesIO(u.read()))
    
def get_manifest_item(manifest, name):
    data = TextIOWrapper(BytesIO(manifest.read(name + ".txt")), 
                         encoding="utf-8", line_buffering=True)
    return pd.read_csv(data)

In [11]:
manifest = get_manifest()

In [12]:
# Stop times for each trip and stop
stop_times = get_manifest_item(manifest, "stop_times")
trips = get_manifest_item(manifest, "trips")

# Add the route_ids and trip_headsigns
stop_times = pd.merge(stop_times[["trip_id", "stop_sequence", "arrival_time"]],
                      trips[["trip_id", "route_id", "trip_headsign"]],
                      on="trip_id")
stop_times.head()

/Users/brian/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/brian/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,trip_id,stop_sequence,arrival_time,route_id,trip_headsign
0,Logan-22-Weekday-trip,1,08:00:00,Logan-22,Loop
1,Logan-22-Weekday-trip,2,08:04:00,Logan-22,Loop
2,Logan-22-Weekday-trip,3,08:09:00,Logan-22,Loop
3,Logan-22-Weekday-trip,4,08:12:00,Logan-22,Loop
4,Logan-22-Weekday-trip,5,08:17:00,Logan-22,Loop


Let's merge the schedule information with the observed data. Join the rows together wherever the trip_id and stop_sequence are the same.

If you're familiar with the notion of joins in relational databases like PostgreSQL, this is the same thing. Specifically, it's an inner join--it will drop rows that don't have a match on both the left and the right.

In [13]:
joined = pd.merge(stops, stop_times, on=["trip_id", "stop_sequence"])
joined.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp,arrival_time,route_id,trip_headsign
0,35291637,2017-09-13,70105,1,545069DD,42.207489,-71.001732,2017-09-13 04:27:00-04:00,05:22:00,Red,Alewife
1,35291635,2017-09-13,70104,10,5450736F,42.214840,-71.000519,2017-09-13 05:19:53-04:00,05:19:00,Red,Alewife
2,35291506,2017-09-13,70092,60,54507680,42.286430,-71.064484,2017-09-13 05:18:37-04:00,05:17:00,Red,Alewife
3,35291507,2017-09-13,70092,60,54507681,42.286430,-71.064484,2017-09-13 05:33:21-04:00,05:31:00,Red,Alewife
4,35291514,2017-09-13,70063,10,545070BD,42.396351,-71.136078,2017-09-13 05:18:40-04:00,05:18:00,Red,Ashmont


Since stops are recurring, scheduled arrival times are recorded as wallclock times in the format `hh:mm:ss`. Since they are anchored to the start of the day, the hour can be greater than 24.

In [14]:
from datetime import datetime, timedelta
import pytz

timezone = pytz.timezone("US/Eastern")

def convert_clock_time(row):
    y, M, d = map(int, row.trip_start.split("-"))
    dt = timezone.localize(datetime(y, M, d))
    h, m, s = map(int, row.arrival_time.split(":", 2))
    # This is here to avoid DST issues
    if h >= 24:
        dt += timedelta(days=1)
        h %= 24
    return dt.replace(hour=h, minute=m, second=s)

Now let's use the function to convert the scheduled arrival time to a timestamp, using the trip start date.

In [15]:
joined["scheduled_arrival_time"] = joined.apply(convert_clock_time, axis=1)
joined.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp,arrival_time,route_id,trip_headsign,scheduled_arrival_time
0,35291637,2017-09-13,70105,1,545069DD,42.207489,-71.001732,2017-09-13 04:27:00-04:00,05:22:00,Red,Alewife,2017-09-13 05:22:00-04:00
1,35291635,2017-09-13,70104,10,5450736F,42.214840,-71.000519,2017-09-13 05:19:53-04:00,05:19:00,Red,Alewife,2017-09-13 05:19:00-04:00
2,35291506,2017-09-13,70092,60,54507680,42.286430,-71.064484,2017-09-13 05:18:37-04:00,05:17:00,Red,Alewife,2017-09-13 05:17:00-04:00
3,35291507,2017-09-13,70092,60,54507681,42.286430,-71.064484,2017-09-13 05:33:21-04:00,05:31:00,Red,Alewife,2017-09-13 05:31:00-04:00
4,35291514,2017-09-13,70063,10,545070BD,42.396351,-71.136078,2017-09-13 05:18:40-04:00,05:18:00,Red,Ashmont,2017-09-13 05:18:00-04:00


Let's add a column with the delay:

In [16]:
joined["delay"] = joined.timestamp - joined.scheduled_arrival_time
joined.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp,arrival_time,route_id,trip_headsign,scheduled_arrival_time,delay
0,35291637,2017-09-13,70105,1,545069DD,42.207489,-71.001732,2017-09-13 04:27:00-04:00,05:22:00,Red,Alewife,2017-09-13 05:22:00-04:00,-1 days +23:05:00
1,35291635,2017-09-13,70104,10,5450736F,42.214840,-71.000519,2017-09-13 05:19:53-04:00,05:19:00,Red,Alewife,2017-09-13 05:19:00-04:00,00:00:53
2,35291506,2017-09-13,70092,60,54507680,42.286430,-71.064484,2017-09-13 05:18:37-04:00,05:17:00,Red,Alewife,2017-09-13 05:17:00-04:00,00:01:37
3,35291507,2017-09-13,70092,60,54507681,42.286430,-71.064484,2017-09-13 05:33:21-04:00,05:31:00,Red,Alewife,2017-09-13 05:31:00-04:00,00:02:21
4,35291514,2017-09-13,70063,10,545070BD,42.396351,-71.136078,2017-09-13 05:18:40-04:00,05:18:00,Red,Ashmont,2017-09-13 05:18:00-04:00,00:00:40


What was the average delay for the Red line?

In [17]:
joined.query('route_id == "Red"').delay.mean()

Timedelta('0 days 00:05:12.113593')

How about some readable stop names?

In [18]:
stop_info = get_manifest_item(manifest, "stops")
stop_info.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding
0,Wareham Village,NaN,Wareham Village,NaN,41.758333,-70.714722,NaN,NaN,0,NaN,1
1,Buzzards Bay,NaN,Buzzards Bay,NaN,41.744805,-70.616226,NaN,NaN,0,NaN,1
2,Hyannis,NaN,Hyannis,NaN,41.660225,-70.276583,NaN,NaN,0,NaN,1
3,Logan-E,NaN,Logan Airport Terminal E,NaN,42.370022,-71.020754,NaN,NaN,0,NaN,1
4,Logan-Subway,NaN,Airport Subway Station,NaN,42.374699,-71.029831,NaN,NaN,0,NaN,1


In [19]:
stops_named = pd.merge(joined, stop_info[["stop_id", "stop_name"]], on=["stop_id"])
stops_named.head()

,trip_id,trip_start,stop_id,stop_sequence,vehicle_id,lat,lon,timestamp,arrival_time,route_id,trip_headsign,scheduled_arrival_time,delay,stop_name
0,35291637,2017-09-13,70105,1,545069DD,42.207489,-71.001732,2017-09-13 04:27:00-04:00,05:22:00,Red,Alewife,2017-09-13 05:22:00-04:00,-1 days +23:05:00,Braintree
1,35291639,2017-09-13,70105,1,545075E8,42.207489,-71.001556,2017-09-13 05:18:18-04:00,05:32:00,Red,Alewife,2017-09-13 05:32:00-04:00,-1 days +23:46:18,Braintree
2,35291648,2017-09-13,70105,1,54507692,42.207489,-71.001556,2017-09-13 05:42:32-04:00,05:53:00,Red,Alewife,2017-09-13 05:53:00-04:00,-1 days +23:49:32,Braintree
3,35291804,2017-09-13,70105,1,545069DE,42.207489,-71.001732,2017-09-13 05:46:09-04:00,06:00:00,Red,Alewife,2017-09-13 06:00:00-04:00,-1 days +23:46:09,Braintree
4,35291662,2017-09-13,70105,1,54507694,42.207489,-71.001556,2017-09-13 05:57:14-04:00,06:07:00,Red,Alewife,2017-09-13 06:07:00-04:00,-1 days +23:50:14,Braintree
